In [1]:
pip install gspread oauth2client

   ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
   ----------------------------------- ---- 51.2/57.5 kB 1.3 MB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.5 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 57.5/57.5 kB 303.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ------------------------- -------------- 61.4/98.2 kB 3.4 MB/s eta 0:00:01
   ------------------------------------- -- 92.2/98.2 kB 880.9 kB/s eta 0:00:01
   ------------------------------------- -- 92.2/98.2 kB 880.9 kB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 470.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/193.7 kB ? eta -:--:--
   ------------ --------------------------- 61.4/193.7 kB 1.7 MB/s eta 0:00:01
   ---------------------- --------------- 112.6/193.7 kB 939.4 kB/s eta 0:00:01
   ------------------------ ------------- 122.9/193.7 kB 804.6 kB/s eta 0:00

In [2]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import schedule
import time

In [ ]:
# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('path/to/your/credentials.json', scope)

# Authorize the client
client = gspread.authorize(creds)

# Function to copy rows from one worksheet to another
def copy_rows(src_worksheet, dest_worksheet, start_row=2):
    rows = src_worksheet.get_all_values()[start_row-1:]  # Get all rows excluding header
    if rows:
        dest_worksheet.append_rows(rows)

# Function to perform the data copying tasks
def sync_sheets():
    # Open the sheets
    warehouse_inventory = client.open("Warehouse Inventory")
    dclm_sheet = client.open("DCLM google sheet")
    dc_returns_tracker = client.open("DC Returns Tracker")
    kawangware_returns_tracker = client.open("Kawangware Returns tracker")

    # Define worksheets
    warehouse_damages = warehouse_inventory.worksheet("Damages")
    warehouse_shipped = warehouse_inventory.worksheet("Shipped")
    warehouse_returns = warehouse_inventory.worksheet("Returns")

    dclm_shipped = dclm_sheet.worksheet("shipped")

    dc_returns = dc_returns_tracker.worksheet("Returns")
    dc_returns_summary = dc_returns_tracker.worksheet("Returns Summary")

    kawangware_returns_sheets = [kawangware_returns_tracker.worksheet("Kawangware returns"),
                                 kawangware_returns_tracker.worksheet("Donholm Returns"),
                                 kawangware_returns_tracker.worksheet("Ruiru returns")]

    # Copy all rows from Kawangware Returns Tracker to DC Returns Tracker
    for sheet in kawangware_returns_sheets:
        copy_rows(sheet, dc_returns)

    # Copy all rows from Returns Summary of DC Returns Tracker to Returns of Warehouse Inventory
    copy_rows(dc_returns_summary, warehouse_returns)

    # Copy all rows from Damages worksheet of Damages Tracker yo Damages worksheet of Warehouse Inventory
    copy_rows(Damages_damages, warehouse_damages)

    # Copy all rows from Shipped worksheet of DCLM to Shipped of Warehouse Inventory
    copy_rows(dclm_shipped, warehouse_shipped)

    print("Warehouse inventory updated successfully!")

# Schedule the sync_sheets function to run every day at 6 AM
schedule.every().day.at("06:00").do(sync_sheets)

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(1)